In [328]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import math

In [329]:
def tree_parameters(r,dt,u,d):
    pu = (np.exp(r*dt)-d)/(u-d)
    pd = 1-pu
    return pu,pd

In [330]:
#input paramteres
s0 = 100
K = 110
T = 1
r = 0.05
sigma = 0.2
n = 3


In [331]:
dt = T/n
u = np.exp(sigma*np.sqrt(dt))
d = 1/u

In [332]:
#Creating Binomial tree
su = []
sd = []
sud =[]

su.append(np.round(s0*u,2))
sd.append(np.round(s0*d,2))
for i in range(2,n+1):

    su.append(np.round(s0*(u**i),2))
    sd.append(np.round(s0*(d**i),2))

    for j in range(1,i):
        
        u1 = u**(i-j)
        d1 = d**j
        sud.append(np.round(s0*u1*d1,2))




In [333]:
print("su:",su)
print("sd:",sd)
print("sud:",sud)

su: [112.24, 125.98, 141.4]
sd: [89.09, 79.38, 70.72]
sud: [100.0, 112.24, 89.09]


In [334]:
data = np.zeros(shape = (len(su)+1,len(su)))
print(data)
                

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [335]:

j = 0
for l in range(1,len(su)):
    for k in range(1,l+1):
        data[k][l] = sud[j]
        #print("k",k)
        #print("l",l)
        #print("\n")
        j = j+1
    #print("\n")       
m = 1
for i in range(len(su)):
    data[0][i] = su[i]
    data[m][i] = sd[i]
    m = m+1

print(data)

[[112.24 125.98 141.4 ]
 [ 89.09 100.   112.24]
 [  0.    79.38  89.09]
 [  0.     0.    70.72]]


In [336]:
col = []
for m in range(1,len(su)+1):
    col.append("{}{}".format('Nodes',m))

print(col)

['Nodes1', 'Nodes2', 'Nodes3']


In [337]:
df_sp = pd.DataFrame( data, index = range(len(su)+1), columns=col)
print(df_sp)

    

   Nodes1  Nodes2  Nodes3
0  112.24  125.98  141.40
1   89.09  100.00  112.24
2    0.00   79.38   89.09
3    0.00    0.00   70.72


In [338]:
df_option = df_sp.copy()
df_option

,Nodes1,Nodes2,Nodes3
0,112.24,125.98,141.40
1,89.09,100.00,112.24
2,0.00,79.38,89.09
3,0.00,0.00,70.72


In [339]:
discount= np.exp(-r*T)
pu,pd =tree_parameters(r,dt,u,d)

In [323]:

def call_option(discount, pu,pd,df_option):
    for i in range(len(df_option.index.values)):
        df_option.iloc[i][-1] = np.round(max(df_option.iloc[i][-1]-K,0),2)
    #print( df_option)
    k =len(df_option.columns.values)-2 #k =3
    l = len(df_option.index.values)-1
    for i in df_option.columns.values[-2::-1]:
        #print("i",i)
        for j in range(0,l):
            #print("j",j)
            df_option.iloc[j][i] = discount*((pu*df_option.iloc[j][k])+(pd*df_option.iloc[j+1][k]))
        k= k-1
        #print("K",k)
        l= l-1
        #print("\n")

    c0 = discount*((pu*df_option.iloc[0][0])+(pd*df_option.iloc[1][0]))
    return c0

In [340]:
def put_option(discount, pu,pd,df_option):
    for i in range(len(df_option.index.values)):
        df_option.iloc[i][-1] = np.round(max(K-df_option.iloc[i][-1],0),2)
        #print(df_option)
    k =len(df_option.columns.values)-2
    l = len(df_option.index.values)-1
    for i in df_option.columns.values[-2:0:-1]:
        for j in range(len(df_option.index.values)-1,-1,-1):
            df_option.iloc[j][i] = discount*((pu*df_option.iloc[j-1][k])+(pd*df_option.iloc[j][k]))
        k= k-1
        l=l-1

    p0 = discount*((pu*df_option.iloc[0][0])+(pd*df_option.iloc[1][0]))
    return p0

In [325]:
print(call_option(discount,pu,pd,df_option))

70.02728458333993


In [341]:
print(put_option(discount, pu,pd,df_option))

   Nodes1  Nodes2  Nodes3
0  112.24  125.98    0.00
1   89.09  100.00  112.24
2    0.00   79.38   89.09
3    0.00    0.00   70.72
   Nodes1  Nodes2  Nodes3
0  112.24  125.98    0.00
1   89.09  100.00    0.00
2    0.00   79.38   89.09
3    0.00    0.00   70.72
   Nodes1  Nodes2  Nodes3
0  112.24  125.98    0.00
1   89.09  100.00    0.00
2    0.00   79.38   20.91
3    0.00    0.00   70.72
   Nodes1  Nodes2  Nodes3
0  112.24  125.98    0.00
1   89.09  100.00    0.00
2    0.00   79.38   20.91
3    0.00    0.00   39.28
96.71951274630025
